# One Hot Encoding! 

In [1]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-31203ca8-2dea-4494-9f15-386a1c7c4d52;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (64ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (25ms)
:: resolution report :: resolve 638ms :: artifacts dl 92ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/13 02:01:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 02:01:37 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


## Read in filter df and onehot

In [2]:
#try to filter motif 5, if not we may have an issue
start_time = time.time()
print(start_time)
df_join_oh = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filters_all/one_hot").cache()
print("read time join: "+ str(time.time() - start_time))

1681332754.598298


read time join: 6.217851638793945


In [3]:
start_time = time.time()
print(start_time)
print(df_join_oh.count())
print("count time join: "+ str(time.time() - start_time))

1681332760.8208025


2299172
count time join: 9.127640724182129


#### check the vitals of the df to make sure the filtering didn't screw anything up

In [6]:
df_join_oh.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)
 |-- relationship: array (nullable = true)
 |    |-- element: string (containsNull = tru

In [9]:
df_join_oh.select('parent_image_path').distinct().show()

+-----------------+
|parent_image_path|
+-----------------+
|             None|
|            other|
|        csrss.exe|
|      svchost.exe|
|          cmd.exe|
|      conhost.exe|
| GoogleUpdate.exe|
+-----------------+



In [11]:
df_join_oh.select('image_path').distinct().show()

+--------------------+
|          image_path|
+--------------------+
|                None|
|          python.EXE|
|          python.exe|
|        Explorer.EXE|
|       taskhostw.exe|
|               other|
|            PING.EXE|
|     geckodriver.exe|
|           csrss.exe|
|         svchost.exe|
|             cmd.exe|
|         conhost.exe|
|         firefox.exe|
|    GoogleUpdate.exe|
| CompatTelRunner.exe|
|              System|
|backgroundTaskHos...|
+--------------------+



In [12]:
df_join_oh.select('file_path_ext').distinct().show()

+-------------+
|file_path_ext|
+-------------+
|         .ini|
|         None|
|          .pf|
|        other|
|         .pyc|
|         .tmp|
|         .bat|
|         .exe|
|         .dll|
|        empty|
+-------------+



#### now start one hot encoding

In [41]:
dict_mapping = {}
list_sparse = []

In [42]:
#for all columns to one hot, one hot, preserve mapping
start_time = time.time()
print(start_time)

for colm in ['file_path_ext','image_path',
                                    'parent_image_path','object','action',"timestamp_bins"]:
    print("\n"+colm+ " start")
    df_join_oh, dict_mapping, list_sparse = oneHotCol(df_join_oh,colm, dict_mapping, list_sparse)
    print(colm + " one hot time: " + str(time.time()-start_time))


1681263954.19483

file_path_ext start
file_path_ext one hot time: 0.18106818199157715

image_path start


image_path one hot time: 2.8104522228240967

parent_image_path start


parent_image_path one hot time: 4.666041135787964

object start


object one hot time: 6.730767488479614

action start


action one hot time: 8.581156015396118

timestamp_bins start


timestamp_bins one hot time: 11.233512878417969


In [43]:
#first count onehot
start_time = time.time()
print(start_time)
print(df_join_oh.count())
print("count time join: "+ str(time.time() - start_time))

1681263965.4328718


2299172
count time join: 2.7587552070617676


In [44]:
#now drop all the irrlevant columns
keep_cols = ['relationship']
drop_cols = [col for col in df_join_oh.columns  
             if col not in list_sparse and col not in keep_cols]

#i want to drop any columnn not in the column list or is the malicious column
df_join_oh = df_join_oh.drop(*drop_cols).cache()

In [45]:
#first count onehot
start_time = time.time()
print(start_time)
print(df_join_oh.count())
print("count time join: "+ str(time.time() - start_time))

1681263968.21468


2299172
count time join: 3.3776869773864746


In [35]:
df_join_oh.printSchema()

root
 |-- relationship: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- file_path_ext_sparse: vector (nullable = true)
 |-- image_path_sparse: vector (nullable = true)
 |-- parent_image_path_sparse: vector (nullable = true)
 |-- object_sparse: vector (nullable = true)
 |-- action_sparse: vector (nullable = true)
 |-- timestamp_bins_sparse: vector (nullable = true)



In [46]:
#combine relationship
df_join_oh = df_join_oh.withColumn('relationship', concat_ws('::',col('relationship')))

In [47]:
#first count onehot
start_time = time.time()
print(start_time)
print(df_join_oh.count())
print("count time join: "+ str(time.time() - start_time))

1681263979.3015542
2299172
count time join: 0.0707082748413086


In [6]:
df_join_oh.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,file_path_ext,motif,timestamp_bins,relationship
0,1245541090864-4,e1,0005435a-f24d-4abe-9795-ddab0f2d729d,4fc0c60c-ff08-4373-99c7-f3896e96759a,2019-09-23 17:22:43.908,FILE,READ,SysClient0874.systemia.com,None,None,...,\Device\HarddiskVolume1\Program Files\VMware\V...,None,None,None,None,0,empty,4,NaN,"[0005435a-f24d-4abe-9795-ddab0f2d729d, 4fc0c60..."


In [49]:
#show dict_map
dict_mapping

{'file_path_ext': ['None',
  'empty',
  '.dll',
  'other',
  '.pyc',
  '.pf',
  '.ini',
  '.tmp',
  '.exe',
  '.bat'],
 'image_path': ['svchost.exe',
  'other',
  'None',
  'python.exe',
  'System',
  'firefox.exe',
  'GoogleUpdate.exe',
  'cmd.exe',
  'backgroundTaskHost.exe',
  'conhost.exe',
  'csrss.exe',
  'Explorer.EXE',
  'taskhostw.exe',
  'PING.EXE',
  'geckodriver.exe',
  'CompatTelRunner.exe',
  'python.EXE'],
 'parent_image_path': ['None',
  'svchost.exe',
  'other',
  'GoogleUpdate.exe',
  'conhost.exe',
  'cmd.exe',
  'csrss.exe'],
 'object': ['FILE', 'PROCESS', 'FLOW', 'SHELL'],
 'action': ['READ',
  'OPEN',
  'MODIFY',
  'INFO',
  'CREATE',
  'WRITE',
  'DELETE',
  'RENAME',
  'COMMAND'],
 'timestamp_bins': ['0.0', '1.0', '2.0', '4.0', '3.0']}

In [50]:
#write this filter for later use
#write to parquet and unpersist
start_time = time.time()
print(start_time)
df_join_oh.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/filters_all/join_oh_enc")
print("write time: "+ str(time.time() - start_time))

1681264516.3864973


write time: 24.21475124359131


In [8]:
#clear what you don't need
#dict_mapping.clear()
df_join_oh.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double, relationship: array<string>]

### Read in trace six and start the encoding process. 

#### Split motif 6 into quarters and write into S3. 

In [1]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-648a090e-6f7b-4fcb-b7a0-11286cb3d19f;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 103ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/13 02:46:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 02:46:51 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/filtered/6").cache()
print("read time 6: "+ str(time.time() - start_time))

1681354013.0282285


read time 6: 5.514738321304321


In [3]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6.count())
print("count time m6: "+ str(time.time() - start_time))

1681354018.547565


766213434
count time m6: 300.44925022125244


In [4]:
#grab distinct traces. 
start_time = time.time()
print(start_time)
splits = motif_6.select('Trace').distinct().randomSplit([.25, .25, .25, .25], 42)
print("split time: "+ str(time.time() - start_time))

1681354319.0027528
split time: 0.07036209106445312


In [5]:
#now count each one and write to S3
#grab distinct traces. 
start_time = time.time()
print(start_time)
print(splits[0].count())
print("count: "+ str(time.time() - start_time))

1681354319.0774043


31924209
count: 19.311486959457397


In [6]:
#now count each one and write to S3
#grab distinct traces. 
start_time = time.time()
print(start_time)
print(splits[1].count())
print("count: "+ str(time.time() - start_time))

1681354338.394129


31916718
count: 17.87901997566223


In [7]:
#now count each one and write to S3
#grab distinct traces. 
start_time = time.time()
print(start_time)
print(splits[2].count())
print("count: "+ str(time.time() - start_time))

1681354356.2778778


31926117
count: 17.895513772964478


In [8]:
#now count each one and write to S3
#grab distinct traces. 
start_time = time.time()
print(start_time)
print(splits[3].count())
print("count: "+ str(time.time() - start_time))

1681354374.1778948


31935195
count: 17.744043588638306


In [9]:
#now, union 3 dfs
start_time = time.time()
split_61 = splits[1].union(splits[2]).union(splits[3])
print("split time: "+ str(time.time() - start_time))

split time: 0.024806976318359375


In [10]:
#count
start_time = time.time()
print(start_time)
print(split_61.count())
print("count: "+ str(time.time() - start_time))

1681354391.9565582


95770551
count: 51.540607929229736


In [33]:
split_61.limit(5).show()

+--------+
|   Trace|
+--------+
|100042-6|
|100137-6|
|100145-6|
|100186-6|
|100242-6|
+--------+



In [12]:
filter_trace = splits[0].select("Trace")

In [19]:
start = time.time()
print(start)
motif_6_1 = motif_6.join(broadcast(filter_trace), motif_6.Trace == filter_trace.Trace)
print("inner join: "+ str(time.time() - start))

1681355588.2900038
inner join: 0.016067981719970703


In [ ]:
#now left anit join for split 6-1
start = time.time()
print(start)
motif_6_1 = motif_6.filter(col('Trace').isin(broadcast(splits[0].Trace)))
print("antijoin: "+ str(time.time() - start))

In [36]:
#now left anit join for split 6-1
start = time.time()
print(start)
motif_6_1 = motif_6.join(broadcast(split_61), ["Trace"], "leftanti").cache()
print("antijoin: "+ str(time.time() - start))

1681353953.163909
antijoin: 0.020625829696655273


In [20]:
#count
start_time = time.time()
print(start_time)
print(motif_6_1.count())
print("count: "+ str(time.time() - start_time))

1681355606.1192946


191545254
count: 113.22172784805298


In [26]:
motif_6_1.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)
 |-- Trace: string (nullable = true)



In [27]:
motif_6_1 = motif_6_1.toDF("Trace", "event",
                           'src','dst','timestamp',
                          'object','action','hostname','user_name','privileges',
                          'image_path','parent_image_path','new_path','file_path','direction','logon_id','requesting_domain',
                          'requesting_user','malicious','file_path_ext','motif','timestamp_bins','Trace2')

In [28]:
#count
start_time = time.time()
print(start_time)
print(motif_6_1.count())
print("count: "+ str(time.time() - start_time))

1681356493.8392742


191545254
count: 98.75374507904053


In [29]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_1.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-1")
print("write time: "+ str(time.time() - start_time))

1681356592.5977156


write time: 549.1572496891022


In [30]:
motif_6_1.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double, Trace2: string]

In [31]:
splits[0].unpersist()

DataFrame[Trace: string]

In [32]:
split_61.unpersist()

DataFrame[Trace: string]

In [33]:
#now do 6-2
filter_trace = splits[1].select("Trace").cache()

In [34]:
start = time.time()
print(start)
motif_6_2 = motif_6.join(broadcast(filter_trace),['Trace'] ,'inner').cache()
print("inner join: "+ str(time.time() - start))

1681357141.8116884
inner join: 0.03121209144592285


In [35]:
#count
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681357141.8470972


191521374
count: 213.85586023330688


In [36]:
motif_6_2.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)



In [37]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-2")
print("write time: "+ str(time.time() - start_time))

1681357541.0932312


write time: 507.204354763031


In [38]:
motif_6_2.unpersist()
splits[1].unpersist()

DataFrame[Trace: string]

In [39]:
#now do 6-3
filter_trace = splits[2].select("Trace").cache()

In [40]:
start = time.time()
print(start)
motif_6_3 = motif_6.join(broadcast(filter_trace),['Trace'] ,'inner').cache()
print("inner join: "+ str(time.time() - start))

1681358048.359222
inner join: 0.0263979434967041


In [41]:
#count
start_time = time.time()
print(start_time)
print(motif_6_3.count())
print("count: "+ str(time.time() - start_time))

1681358048.3898547


191521290
count: 182.17591643333435


In [42]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-3")
print("write time: "+ str(time.time() - start_time))

1681358230.5704741


write time: 529.6540560722351


In [43]:
motif_6_3.unpersist()
splits[2].unpersist()

DataFrame[Trace: string]

In [44]:
#now do 6-4
filter_trace = splits[3].select("Trace").cache()

In [45]:
start = time.time()
print(start)
motif_6_4 = motif_6.join(broadcast(filter_trace),['Trace'] ,'inner').cache()
print("inner join: "+ str(time.time() - start))

1681358760.269228
inner join: 0.024412870407104492


In [ ]:
splits[3].limit(5).show()

In [ ]:
#count
start_time = time.time()
print(start_time)
print(motif_6_4.count())
print("count: "+ str(time.time() - start_time))

In [47]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-4")
print("write time: "+ str(time.time() - start_time))

1681358949.682862


write time: 502.7249984741211


In [48]:
motif_6_4.unpersist()
splits[3].unpersist()
motif_6.unpersist()
splits.clear()

#### Motif 6: Start with 2

In [9]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

In [4]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-2").cache()
print("read time: "+ str(time.time() - start_time))

1681361159.2555141
read time: 2.9028491973876953


In [5]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count time m6: "+ str(time.time() - start_time))

1681361169.427934


191521374
count time m6: 93.42988872528076


##### One-Hot with broadcast join.  

In [6]:
#read in motif6
start_time = time.time()
print(start_time)
df_join_oh = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filters_all/join_oh_enc").cache()
print("read join oh: "+ str(time.time() - start_time))

1681361262.862513
read join oh: 0.395998477935791


In [8]:
#first count onehot
start_time = time.time()
print(start_time)
print(df_join_oh.count())
print("count join oh: "+ str(time.time() - start_time))

1681361268.175701
2299172
count join oh: 0.08635616302490234


In [9]:
motif_6_2.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)



In [ ]:
motif_6_2.limit(1).toPandas()

In [10]:
#create relationship for motif_6
#now add relationship column and combine
start_time = time.time()
motif_6_2 = motif_6_2.withColumn("relationship", array("src", "dst"))\
                    .withColumn("relationship", concat_ws("::",col("relationship"))).cache()
print("relationship: "+ str(time.time() - start_time))

relationship: 0.12168240547180176


In [11]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count time: "+ str(time.time() - start_time))

1681361292.5594885


191521374
count time: 94.43246579170227


In [12]:
#execute broadcast join and count
# Explain broadcast join
start_time = time.time()
print(start_time)
motif_6_2 = motif_6_2.join(broadcast(df_join_oh),["relationship"])
print('join: ' + str(time.time()-start_time))

1681361386.9966998
join: 0.03528118133544922


In [13]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count time: "+ str(time.time() - start_time))

1681361387.0361595


191521374
count time: 17.651797771453857


In [12]:
motif_6_1.limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,relationship,Trace,event,src,dst,timestamp,object,action,hostname,user_name,...,file_path_ext,motif,timestamp_bins,Trace2,file_path_ext_sparse,image_path_sparse,parent_image_path_sparse,object_sparse,action_sparse,timestamp_bins_sparse
0,05e6d567-0157-40ef-9661-69fbb76264f1::57060c5f...,1632088059046-6,e1,05e6d567-0157-40ef-9661-69fbb76264f1,57060c5f-266c-491c-a1d6-b91735388a97,2019-09-24 14:57:40.255,FILE,READ,SysClient0771.systemia.com,None,...,.dll,6,NaN,1632088059046-6,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)"


In [ ]:
df_join_oh.unpersist()

##### now drop cols and do everything else

In [6]:
list_sparse = ['file_path_ext_sparse','image_path_sparse',
              'parent_image_path_sparse','object_sparse',
              'action_sparse','timestamp_bins_sparse']

In [21]:
#drop cols
start_time = time.time()
print(start_time)
#Generate a list of columns to drop
keep_cols = ['Trace','event', 'malicious','motif']
drop_cols = [col for col in motif_6.columns  
             if col not in keep_cols and col not in list_sparse]

#i want to drop any columnn not in the column list or is the malicious column
motif_6 = motif_6.drop(*drop_cols).cache()
print("drop time: "+ str(time.time() - start_time))

1681333587.080506
drop time: 0.04308795928955078


In [22]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6.count())
print("count time m6: "+ str(time.time() - start_time))

1681333592.3526347


766213434
count time m6: 768.0026433467865


In [15]:
#next vector assemble
start_time = time.time()
print(start_time)
#assemble vectors for all sparse columns - this might be enough for our ML algorithms
assembler = VectorAssembler(inputCols=list_sparse, 
                        outputCol="final_vec")
motif_6_2 = assembler.transform(motif_6_2).cache()
print("vec assemble: "+ str(time.time() - start_time))

1681361404.7000308
vec assemble: 0.23842120170593262


In [16]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681361404.9440928


191521374
count: 248.51026439666748


In [17]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-2")
print("write time: "+ str(time.time() - start_time))

1681361653.4597707


write time: 636.6844384670258


In [18]:
motif_6_2.unpersist()

DataFrame[relationship: string, Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double, file_path_ext_sparse: vector, image_path_sparse: vector, parent_image_path_sparse: vector, object_sparse: vector, action_sparse: vector, timestamp_bins_sparse: vector, final_vec: vector]

In [1]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ca82569d-aa0c-4c8f-a8cb-ee7ce918a89c;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 101ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/13 14:49:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 14:49:06 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/13 14:49:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/6-2").cache()
print("read time: "+ str(time.time() - start_time))

1681393321.5076973


read time: 7.95035982131958


In [3]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681393335.1687183


191521374
count: 431.4762399196625


In [4]:
#Let's drop some cols and make this easier
motif_6_2.printSchema()

root
 |-- relationship: string (nullable = true)
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)
 |-- file_path_ext_sparse: vector (nullable 

In [7]:
list_sparse = ['file_path_ext_sparse','image_path_sparse',
              'parent_image_path_sparse','object_sparse',
              'action_sparse','timestamp_bins_sparse']

In [7]:
#Generate a list of columns to drop
keep_cols = ['Trace','event','malicious','motif']
drop_cols = [col for col in motif_6_2.columns  
             if col not in keep_cols and col not in list_sparse]
#drop_cols = broadcast(drop_cols)
keep_cols = keep_cols + list_sparse

In [15]:
#drop cols again
start_time = time.time()
print(start_time)
#i want to drop any columnn not in the column list or is the malicious column
#motif_6_2 = motif_6_2.drop(*broadcast(drop_cols)).cache()
motif_6_2 = motif_6_2.select(*keep_cols).cache() # .drop(*broadcast(drop_cols)).cache()
print("drop time: "+ str(time.time() - start_time))

1681392116.1509392
drop time: 0.08568501472473145


In [16]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681392126.2599192


191521374
count: 179.33009934425354


In [17]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

1681392305.5946138


write time: 420.9260139465332


In [18]:
motif_6_2.unpersist()

DataFrame[Trace: string, event: string, malicious: int, motif: int, file_path_ext_sparse: vector, image_path_sparse: vector, parent_image_path_sparse: vector, object_sparse: vector, action_sparse: vector, timestamp_bins_sparse: vector]

In [2]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

1681397383.5594716


read time: 5.600620985031128


In [3]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681397391.0690439


191521374
count: 92.0152645111084


In [6]:
motif_6_2.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- motif: integer (nullable = true)
 |-- file_path_ext_sparse: vector (nullable = true)
 |-- image_path_sparse: vector (nullable = true)
 |-- parent_image_path_sparse: vector (nullable = true)
 |-- object_sparse: vector (nullable = true)
 |-- action_sparse: vector (nullable = true)
 |-- timestamp_bins_sparse: vector (nullable = true)
 |-- final_vec: vector (nullable = true)



In [7]:
motif_6_2.limit(1).toPandas()

,Trace,event,malicious,motif,file_path_ext_sparse,image_path_sparse,parent_image_path_sparse,object_sparse,action_sparse,timestamp_bins_sparse,final_vec
0,60129561799-6,e1,0,6,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [5]:
#now we need to recreate the final vec
#next vector assemble
start_time = time.time()
print(start_time)
list_sparse = ['file_path_ext_sparse','image_path_sparse',
              'parent_image_path_sparse','object_sparse',
              'action_sparse','timestamp_bins_sparse']
#assemble vectors for all sparse columns - this might be enough for our ML algorithms
assembler = VectorAssembler(inputCols=list_sparse, 
                        outputCol="final_vec")
motif_6_2 = assembler.transform(motif_6_2).cache()
print("vec assemble: "+ str(time.time() - start_time))

1681397504.0507534
vec assemble: 0.26070070266723633


In [8]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681397549.4356234


191521374
count: 245.4598970413208


In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

1681397794.9003763


In [ ]:
motif_6_2.unpersist()

In [ ]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

In [ ]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [13]:
#drop cols again
start_time = time.time()
print(start_time)
#i want to drop any columnn not in the column list or is the malicious column
#motif_6_2 = motif_6_2.drop(*broadcast(drop_cols)).cache()
motif_6_2 = motif_6_2.select(*keep_cols).cache() # .drop(*broadcast(drop_cols)).cache()
print("drop time: "+ str(time.time() - start_time))

1681395739.5157473
drop time: 0.08246755599975586


In [14]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681395756.3650596


191521374
count: 70.03514313697815


In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.unpersist()

In [ ]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

In [ ]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [15]:
start_time = time.time()
print(start_time)
#turn into string
motif_6_2 = motif_6_2.withColumn("vec2string", binary_to_string_array_udf("final_vec")).cache()
print("turn into a string: "+ str(time.time() - start_time))

1681395855.583914
turn into a string: 0.08121180534362793


In [16]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

1681395859.7926571


ERROR:root:KeyboardInterrupt while sending command.               (0 + 32) / 77]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.unpersist()

In [ ]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

In [ ]:
#first count onehot
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.printSchema()

In [ ]:
motif_6_2.limit(1).toPandas()

In [ ]:
#add mal trace 
w = Window.partitionBy("Trace")
motif_6_2 = motif_6_2.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()

In [ ]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.unpersist()

In [ ]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.printSchema()

In [ ]:
motif_6_2.limit(1).toPandas()

In [ ]:
#Generate a list of columns to drop
keep_cols = ['mal_trace','motif','Trace','event','vec2string']
drop_cols = [col for col in motif_6_2.columns  
             if col not in keep_cols]

In [ ]:
#drop cols again
start_time = time.time()
print(start_time)
#i want to drop any columnn not in the column list or is the malicious column
motif_6_2 = motif_6_2.drop(*drop_cols).cache()
print("drop time: "+ str(time.time() - start_time))

In [ ]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.limit(1).toPandas()

In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.unpersist()

In [ ]:
#read in motif6
start_time = time.time()
print(start_time)
motif_6_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2").cache()
print("read time: "+ str(time.time() - start_time))

In [ ]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [ ]:
#first pivot aka transpose and keep all events
motif_6_2 = motif_6_2.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()

In [ ]:
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count: "+ str(time.time() - start_time))

In [ ]:
#then consolidate the columns into a single event sequence
motif_6_2 = motif_6_2.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)','e4_first(vec2string)', 
                   'e5_first(vec2string)', 'e6_first(vec2string)').alias('event_sequence')).cache()

In [ ]:
#count
start_time = time.time()
print(start_time)
print(motif_6_2.count())
print("count time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.limit(5).toPandas()

In [ ]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
motif_6_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motif-subs/enc/6/6-2")
print("write time: "+ str(time.time() - start_time))

In [ ]:
motif_6_2.unpersist()